# Attention : 
# Faire "File -> Save a copy in Drive" avant de commencer à modifier le notebook, sinon vos modifications ne seront pas sauvegardées.


In [ ]:
!wget http://webia.lip6.fr/~dancette/deep-learning/assets/TP3-4/TP3-4.zip
!unzip -j TP3-4.zip

--2020-09-30 17:20:07--  http://webia.lip6.fr/~dancette/deep-learning/assets/TP3-4/TP3-4.zip
Resolving webia.lip6.fr (webia.lip6.fr)... 132.227.201.33
Connecting to webia.lip6.fr (webia.lip6.fr)|132.227.201.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13421167 (13M) [application/zip]
Saving to: ‘TP3-4.zip’

TP3-4.zip           100%[===================>]  12.80M  11.2MB/s    in 1.1s    

2020-09-30 17:20:08 (11.2 MB/s) - ‘TP3-4.zip’ saved [13421167/13421167]

Archive:  TP3-4.zip
  inflating: tme5.py                 
  inflating: mnist.mat               
  inflating: circles.py              
  inflating: circles.mat             


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/git/RDFIA/TME 3-4")
!ls

Mounted at /content/gdrive
 circles.mat
 circles.py
'Compte rendu TME3-4 DJEGHRI_MAMOU.docx'
 mnist.mat
 tme5.py
 TP_3_4_Introduction_aux_réseaux_de_neurones_Sorbonne.ipynb
 tp3-4.pdf
'tp3-4 solutions gradient.pdf'


In [42]:
import math
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%run tme5.py

<Figure size 432x288 with 0 Axes>

# Partie 1 : Forward et Backward manuels

In [43]:
def init_params(nx, nh, ny):
    """
    nx, nh, ny: integers
    out params: dictionnary
    """
    params = {}
    
    # remplir avec les paramètres Wh, Wy, bh, by

    params["Wh"] = torch.normal(mean=0, std=0.3, size=(nh,nx))
                # or torch.randn(nh, nx).normal_(mean=0,std=0.3)
    params["Wy"] = torch.normal(mean=0, std=0.3, size=(ny,nh))
    params["bh"] = torch.normal(mean=0, std=0.3, size=(nh,))
    params["by"] = torch.normal(mean=0, std=0.3, size=(ny,))
    
    return params

In [44]:
def forward(params, X):
    """
    params: dictionnary
    X: (n_batch, dimension)
    """
    bsize = X.size(0)
    nh = params['Wh'].size(0)
    ny = params['Wy'].size(0)
    outputs = {}


    # remplir avec les paramètres X, htilde, h, ytilde, yhat
    
    outputs["X"] = X
    outputs["htilde"] = torch.mm(X,params["Wh"].T) + params["bh"]
    outputs["h"] = torch.tanh(outputs["htilde"])
    outputs["ytilde"] = torch.mm(outputs["h"],params["Wy"].T) + params["by"]
    outputs["yhat"] = torch.exp(outputs["ytilde"]) / torch.sum(torch.exp(outputs["ytilde"]), dim=1, keepdim=True)

    return outputs['yhat'], outputs

In [45]:
def loss_accuracy(Yhat, Y):


    L = 0
    acc = 0

    L = -torch.mean(torch.sum(Y*torch.log(Yhat)))

    _, indsY = torch.max(Y, 1)
    _, indsYhat = torch.max(Yhat, 1)
    acc = (indsY == indsYhat).float().mean()*100

    return L, acc

In [46]:
def backward(params, outputs, Y):
    bsize = Y.shape[0]
    grads = {}

    # remplir avec les paramètres Wy, Wh, by, bh

    # dérivé de L par rapport à ytilde
    grad_L_ytilde = outputs['yhat'] - Y 

    # dérivé de ytilde par rapport à Wh
    grad_ytilde_Wh = outputs['h']
    # dérivé de ytilde par rapport à Bh
    grad_ytilde_bh = 1


    grads["Wy"] = torch.mm(grad_L_ytilde.T, grad_ytilde_Wh)   # dérivé de L par rapport à Wy
    grads["by"] = torch.sum(grad_L_ytilde, dim=0)   # dérivé de L par rapport à by

    # dérivé de y_tilde par rapport à h
    grad_ytilde_h = params['Wy']
    # dérivé de L par rapport à htilde
    grad_h_htilde = torch.mm(grad_L_ytilde,grad_ytilde_h) * (1-torch.pow(grad_ytilde_Wh,2))

    grads["Wh"] = torch.mm(grad_h_htilde, outputs['X'])  # dérivé de L par rapport à Wh
    
    grads["bh"] = torch.sum(grad_h_htilde, dim=0)  # dérivé de L par rapport à bh

    return grads

In [47]:
def sgd(params, grads, eta):
    params["Wh"] = params["Wh"] - eta * grads['Wh']  
    params["Wy"] = params["Wy"] - eta * grads['Wy']  
    params["bh"] = params["bh"] - eta * grads['bh']  
    params["by"] = params["by"] - eta * grads['by']    

    return params

## Algorithme global d'apprentissage (manuel)

In [48]:
# init
data = CirclesData()
data.plot_data()
N = data.Xtrain.shape[0]
Nbatch = 10
nx = data.Xtrain.shape[1]
nh = 10
ny = data.Ytrain.shape[1]
eta = 0.03

params = init_params(nx, nh, ny)

curves = [[],[], [], []]

# epoch
for iteration in range(150):

    # permute
    perm = np.random.permutation(N)
    Xtrain = data.Xtrain[perm, :]
    Ytrain = data.Ytrain[perm, :]


    # batches
    for j in range(N // Nbatch):

        indsBatch = range(j * Nbatch, (j+1) * Nbatch)
        X = Xtrain[indsBatch, :]
        Y = Ytrain[indsBatch, :]

        # écrire l'algorithme d'apprentissage sur le batch (X,Y)
        # en utilisant les fonctions forward, loss_accuracy, backward, sgd 

        Yhat, outputs = forward(params, X)
        loss, acc = loss_accuracy(Yhat, Y)
        grads = backward(params, outputs, Y) 
        params = sgd(params, grads, eta)

    Yhat_train, _ = forward(params, data.Xtrain)
    Yhat_test, _ = forward(params, data.Xtest)
    Ltrain, acctrain = loss_accuracy(Yhat_train, data.Ytrain)
    Ltest, acctest = loss_accuracy(Yhat_test, data.Ytest)
    Ygrid, _ = forward(params, data.Xgrid)  

    title = 'Iter {}: Acc train {:.1f}% ({:.2f}), acc test {:.1f}% ({:.2f})'.format(iteration, acctrain, Ltrain, acctest, Ltest)
    print(title)
    data.plot_data_with_grid(Ygrid, title)

    curves[0].append(acctrain)
    curves[1].append(acctest)
    curves[2].append(Ltrain)
    curves[3].append(Ltest)

fig = plt.figure()
plt.plot(curves[0], label="acc. train")
plt.plot(curves[1], label="acc. test")
plt.plot(curves[2], label="loss train")
plt.plot(curves[3], label="loss test")
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Partie 2 : Simplification du backward avec `torch.autograd`



In [49]:
def init_params(nx, nh, ny):
    """
    nx, nh, ny: integers
    out params: dictionnary
    """
    params = {}
    

    # remplir avec les paramètres Wh, Wy, bh, by
    # activer l'autograd sur les poids du réseau
    
    params["Wh"] = torch.normal(mean=0, std=0.3, size=(nh, nx), requires_grad=True)
    params["Wy"] = torch.normal(mean=0, std=0.3, size=(ny, nh), requires_grad=True)
    params["bh"] = torch.normal(mean=0, std=0.3, size=(nh,), requires_grad=True)
    params["by"] = torch.normal(mean=0, std=0.3, size=(ny,), requires_grad=True)
    

    return params

La fonction `forward` est inchangée par rapport à la partie précédente. 

La fonction `backward` n'est plus utilisée grâce à l'autograd. 

In [50]:
def sgd(params, eta):

    # mettre à jour le contenu de params
    # attention à bien utiliser torch.no_grad()
    # et à remettre les accumulateurs de gradients à zéro
    with torch.no_grad():
        for param in params.values():
          param-= eta*param.grad
          param.grad.zero_()


    return params

## Algorithme global d'apprentissage (avec autograd)

In [51]:
# init
data = CirclesData()
data.plot_data()
N = data.Xtrain.shape[0]
Nbatch = 10
nx = data.Xtrain.shape[1]
nh = 10
ny = data.Ytrain.shape[1]
eta = 0.03

params = init_params(nx, nh, ny)

curves = [[],[], [], []]

# epoch
for iteration in range(150):

    # permute
    perm = np.random.permutation(N)
    Xtrain = data.Xtrain[perm, :]
    Ytrain = data.Ytrain[perm, :]


    # batches
    for j in range(N // Nbatch):

        indsBatch = range(j * Nbatch, (j+1) * Nbatch)
        X = Xtrain[indsBatch, :]
        Y = Ytrain[indsBatch, :]
  
        # écrire l'algorithme d'apprentissage sur le batch (X,Y)
        # en utilisant les fonctions forward, loss_accuracy, sgd 
        # calculer les gradients avec la fonction backward de l'autograd 

        Yhat, outputs = forward(params, X)
        loss, acc = loss_accuracy(Yhat, Y)
        loss.backward()
        params = sgd(params, eta)



    Yhat_train, _ = forward(params, data.Xtrain)
    Yhat_test, _ = forward(params, data.Xtest)
    Ltrain, acctrain = loss_accuracy(Yhat_train, data.Ytrain)
    Ltest, acctest = loss_accuracy(Yhat_test, data.Ytest)
    Ygrid, _ = forward(params, data.Xgrid)  

    title = 'Iter {}: Acc train {:.1f}% ({:.2f}), acc test {:.1f}% ({:.2f})'.format(iteration, acctrain, Ltrain, acctest, Ltest)
    print(title)
    # detach() est utilisé pour détacher les predictions du graphes de calcul autograd
    data.plot_data_with_grid(Ygrid.detach(), title)

    curves[0].append(acctrain)
    curves[1].append(acctest)
    curves[2].append(Ltrain)
    curves[3].append(Ltest)

fig = plt.figure()
plt.plot(curves[0], label="acc. train")
plt.plot(curves[1], label="acc. test")
plt.plot(curves[2], label="loss train")
plt.plot(curves[3], label="loss test")
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Partie 3 : Simplification du forward avec `torch.nn`

`init_params` et `forward` sont supprimés et remplacés par une fonction `init_model` qui déclare l'architecture du modèle et la loss.

In [58]:
def init_model(nx, nh, ny):

    model = torch.nn.Sequential(
        torch.nn.Linear(nx, nh),
        torch.nn.Tanh(),
        torch.nn.Linear(nh, ny),
    )
    # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class 
    loss = torch.nn.CrossEntropyLoss()

    return model, loss

In [65]:
def loss_accuracy(loss, Yhat, Y):

    # faire appel à la fonction de loss

    L = loss(Yhat, Y)
    _, indsYhat = torch.max(Yhat, 1)
    acc = (Y == indsYhat).float().mean()*100

    return L, acc

In [66]:
def sgd(model, eta):

    # mettre à jour les paramètres de model
    # attention à bien utiliser torch.no_grad()
    # et à remettre les accumulateurs de gradients à zéro
    with torch.no_grad():
        for param in model.parameters():
            param -= eta * param.grad
        model.zero_grad()

    return model

## Algorithme global d'apprentissage (avec autograd et les couches `torch.nn`)

In [74]:
# init
data = CirclesData()
data.plot_data()
N = data.Xtrain.shape[0]
Nbatch = 10
nx = data.Xtrain.shape[1]
nh = 10
ny = data.Ytrain.shape[1]
eta = 0.03

model, loss = init_model(nx, nh, ny)

curves = [[],[], [], []]

# epoch
for iteration in range(150):

    # permute
    perm = np.random.permutation(N)
    Xtrain = data.Xtrain[perm, :]
    Ytrain = data.Ytrain[perm, :]
    Ytrain = torch.argmax(Ytrain, dim=1)

    # batches
    for j in range(N // Nbatch):

        indsBatch = range(j * Nbatch, (j+1) * Nbatch)
        X = Xtrain[indsBatch, :]
        Y = Ytrain[indsBatch]
  
        # écrire l'algorithme d'apprentissage sur le batch (X,Y)
        # en utilisant les fonctions loss_accuracy, sgd
        # effectuer le forward en faisant un appel au modèle 
        # calculer les gradients avec la fonction backward de l'autograd 

        Yhat = model(X)
        l, acc = loss_accuracy(loss, Yhat, Y)
        l.backward()
        sgd(model, eta)



    Yhat_train = model(data.Xtrain)
    Yhat_test = model(data.Xtest)

    Ytrain = torch.argmax(data.Ytrain, dim=1)
    Ltrain, acctrain = loss_accuracy(loss, Yhat_train, Ytrain)
    Ytest = torch.argmax(data.Ytest, dim=1)
    Ltest, acctest = loss_accuracy(loss, Yhat_test, Ytest)
    Ygrid = model(data.Xgrid)  

    title = 'Iter {}: Acc train {:.1f}% ({:.2f}), acc test {:.1f}% ({:.2f})'.format(iteration, acctrain, Ltrain, acctest, Ltest)
    print(title) 
    data.plot_data_with_grid(torch.nn.Softmax(dim=1)(Ygrid.detach()), title)

    curves[0].append(acctrain)
    curves[1].append(acctest)
    curves[2].append(Ltrain)
    curves[3].append(Ltest)

fig = plt.figure()
plt.plot(curves[0], label="acc. train")
plt.plot(curves[1], label="acc. test")
plt.plot(curves[2], label="loss train")
plt.plot(curves[3], label="loss test")
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Partie 4 : Simplification de SGD avec `torch.optim`

In [75]:
def init_model(nx, nh, ny, eta):
  
    model = torch.nn.Sequential(
        torch.nn.Linear(nx, nh),
        torch.nn.Tanh(),
        torch.nn.Linear(nh, ny),
    )
    # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class 
    loss = torch.nn.CrossEntropyLoss()
    optim = torch.optim.SGD(model.parameters(), lr=eta)

    return model, loss, optim

La fonction `sgd` est supprimée. À la place, on fera un appel à `optim.zero_grad()` avant de faire le backward et à `optim.step()` après le backward. 

## Algorithme global d'apprentissage (avec autograd, les couches `torch.nn` et `torch.optim`)

In [81]:
# init
data = CirclesData()
data.plot_data()
N = data.Xtrain.shape[0]
Nbatch = 10
nx = data.Xtrain.shape[1]
nh = 10
ny = data.Ytrain.shape[1]
eta = 0.03

model, loss, optim = init_model(nx, nh, ny, eta)

curves = [[],[], [], []]

# epoch
for iteration in range(150):

    # permute
    perm = np.random.permutation(N)
    Xtrain = data.Xtrain[perm, :]
    Ytrain = data.Ytrain[perm, :]
    Ytrain = torch.argmax(Ytrain, dim=1)

    # batches
    for j in range(N // Nbatch):

        indsBatch = range(j * Nbatch, (j+1) * Nbatch)
        X = Xtrain[indsBatch, :]
        Y = Ytrain[indsBatch]

        # écrire l'algorithme d'apprentissage sur le batch (X,Y)
        # en utilisant la fonction loss_accuracy
        # effectuer le forward en faisant un appel au modèle 
        # calculer les gradients avec la fonction backward de l'autograd 
        # Puis une "step" d'optimisation

        Yhat = model(X)
        L= loss(Yhat, Y)

        optim.zero_grad()
        L.backward()
        optim.step()
        


    Yhat_train = model(data.Xtrain)
    Yhat_test = model(data.Xtest)
    Ytrain = torch.argmax(data.Ytrain, dim=1)
    Ytest = torch.argmax(data.Ytest, dim=1)
    Ltrain, acctrain = loss_accuracy(loss, Yhat_train, Ytrain)
    Ltest, acctest = loss_accuracy(loss, Yhat_test, Ytest)
    Ygrid = model(data.Xgrid)  

    title = 'Iter {}: Acc train {:.1f}% ({:.2f}), acc test {:.1f}% ({:.2f})'.format(iteration, acctrain, Ltrain, acctest, Ltest)
    print(title) 
    data.plot_data_with_grid(torch.nn.Softmax(dim=1)(Ygrid.detach()), title)

    curves[0].append(acctrain)
    curves[1].append(acctest)
    curves[2].append(Ltrain)
    curves[3].append(Ltest)

fig = plt.figure()
plt.plot(curves[0], label="acc. train")
plt.plot(curves[1], label="acc. test")
plt.plot(curves[2], label="loss train")
plt.plot(curves[3], label="loss test")
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Partie 5 : MNIST

Reprendre le code d'entraînement précédent en l'appliquant au dataset MNIST.

In [83]:
# init
data = MNISTData()
N = data.Xtrain.shape[0]
Nbatch = 100
nx = data.Xtrain.shape[1]
nh = 100
ny = data.Ytrain.shape[1]
eta = 0.03

model, loss, optim = init_model(nx, nh, ny, eta)


# epoch
for iteration in range(150):

    # permute
    perm = np.random.permutation(N)
    Xtrain = data.Xtrain[perm, :]
    Ytrain = data.Ytrain[perm, :]
    Ytrain = torch.argmax(Ytrain, dim=1)

    # batches
    for j in range(N // Nbatch):

        indsBatch = range(j * Nbatch, (j+1) * Nbatch)
        X = Xtrain[indsBatch, :]
        Y = Ytrain[indsBatch]

        # écrire l'algorithme d'apprentissage sur le batch (X,Y)
        # en utilisant la fonction loss_accuracy
        # effectuer le forward en faisant un appel au modèle 
        # calculer les gradients avec la fonction backward de l'autograd 
        # Puis une "step" d'optimisation

        Yhat = model(X)
        L= loss(Yhat, Y)

        optim.zero_grad()
        L.backward()
        optim.step()
        


    Yhat_train = model(data.Xtrain)
    Yhat_test = model(data.Xtest)
    Ytrain = torch.argmax(data.Ytrain, dim=1)
    Ytest = torch.argmax(data.Ytest, dim=1)
    Ltrain, acctrain = loss_accuracy(loss, Yhat_train, Ytrain)
    Ltest, acctest = loss_accuracy(loss, Yhat_test, Ytest)
    title = 'Iter {}: Acc train {:.1f}% ({:.2f}), acc test {:.1f}% ({:.2f})'.format(iteration, acctrain, Ltrain, acctest, Ltest)
    print(title) 

Iter 0: Acc train 85.0% (0.53), acc test 85.9% (0.51)
Iter 1: Acc train 85.8% (0.49), acc test 86.7% (0.46)
Iter 2: Acc train 89.0% (0.38), acc test 89.4% (0.37)
Iter 3: Acc train 88.5% (0.39), acc test 88.8% (0.38)
Iter 4: Acc train 88.4% (0.40), acc test 89.1% (0.39)
Iter 5: Acc train 88.2% (0.39), acc test 89.1% (0.37)
Iter 6: Acc train 88.1% (0.40), acc test 88.5% (0.39)
Iter 7: Acc train 88.3% (0.38), acc test 88.4% (0.37)
Iter 8: Acc train 89.0% (0.36), acc test 89.5% (0.35)
Iter 9: Acc train 88.6% (0.37), acc test 88.7% (0.36)
Iter 10: Acc train 89.1% (0.37), acc test 89.4% (0.36)
Iter 11: Acc train 89.2% (0.36), acc test 90.0% (0.33)
Iter 12: Acc train 89.6% (0.35), acc test 89.6% (0.34)
Iter 13: Acc train 89.0% (0.37), acc test 89.3% (0.35)
Iter 14: Acc train 88.9% (0.36), acc test 89.4% (0.36)
Iter 15: Acc train 89.7% (0.34), acc test 89.9% (0.34)
Iter 16: Acc train 88.6% (0.37), acc test 88.9% (0.36)
Iter 17: Acc train 88.6% (0.37), acc test 88.5% (0.37)
Iter 18: Acc train 8

# Partie 6: Bonus: SVM


Entrainez un modèle SVM sur le jeu de data Circles.
Le code à remplir est indiqué par des commentaires.


Idées : 
- Essayer d'abord un SVM linéaire (sklearn.svm.LinearSVC dans scikit-learn). Est-ce que cela fonctionne bien ? Pourquoi
- Essayer d'autres kernels (possible avec sklearn.svm.SVC).
Lequel fonctionne le mieux ? Pourquoi ?
- Est-ce que le paramètre C de régularisation à un impact ? Pourquoi ?

In [ ]:
# data
data = CirclesData()
Xtrain = data.Xtrain.numpy()
Ytrain = data.Ytrain[:, 0].numpy()

Xgrid = data.Xgrid.numpy()

Xtest = data.Xtest.numpy()
Ytest = data.Ytest[:, 0].numpy()

def plot_svm_predictions(data, predictions):
      plt.figure(2)
      plt.clf()
      plt.imshow(np.reshape(predictions, (40,40)))
      plt.plot(data._Xtrain[data._Ytrain[:,0] == 1,0]*10+20, data._Xtrain[data._Ytrain[:,0] == 1,1]*10+20, 'bo', label="Train")
      plt.plot(data._Xtrain[data._Ytrain[:,1] == 1,0]*10+20, data._Xtrain[data._Ytrain[:,1] == 1,1]*10+20, 'ro')
      plt.plot(data._Xtest[data._Ytest[:,0] == 1,0]*10+20, data._Xtest[data._Ytest[:,0] == 1,1]*10+20, 'b+', label="Test")
      plt.plot(data._Xtest[data._Ytest[:,1] == 1,0]*10+20, data._Xtest[data._Ytest[:,1] == 1,1]*10+20, 'r+')
      plt.xlim(0,39)
      plt.ylim(0,39)
      plt.clim(0.3,0.7)
      plt.draw()
      plt.pause(1e-3)

In [ ]:
import sklearn.svm

############################
### Votre code ici   #######
### Entrainer le SVM #######
## Voir https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
## et https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
############################

svm = None

###########################

In [ ]:
## Affichage des résultats

Ytest_pred = svm.predict(Xtest)
accuracy = np.sum(Ytest == Ytest_pred) / len(Ytest)
print(f"Accuracy : {100 * accuracy:.2f}")
Ygrid_pred = svm.predict(Xgrid)
plot_svm_predictions(data, Ygrid_pred)